In [4]:
%load_ext autoreload
%autoreload 2
import utils
import json
import sys
# sys.path.append('/home/jupyter/tldr-reddit-summarization/pegasus')
import pegasus
import tensorflow as tf
# import tensorflow_datasets as tfds

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
tf.__version__

'2.1.0'

# Create TFRecords for PEGASUS library

In [6]:
reddit_posts = utils.load_reddit_data_from_json()

0
1000000
2000000
3000000


In [7]:
subreddits, content, summary = utils.extract_tensors(reddit_posts)

In [8]:
features_dataset = tf.data.Dataset.from_tensor_slices({'topics':subreddits, 
                                                       'inputs': content, 
                                                       'targets': summary})

In [10]:
# serialized_features_dataset = features_dataset.map(utils.tf_serialize_example)

# def generator():
#     for features in features_dataset.take(100):
#         yield utils.serialize_example(*features)

# serialized_features_dataset = tf.data.Dataset.from_generator(
#     generator, output_types=tf.string, output_shapes=())

In [14]:
def write_tfrecord_file(dataset, filename):
    serialized_features_dataset = dataset.map(utils.tf_serialize_example)
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_features_dataset)

In [15]:
write_tfrecord_file(features_dataset.take(1000), 'reddit_small.tfrecord')

In [16]:
write_tfrecord_file(features_dataset, 'reddit_all.tfrecord')

## Read TFRecords

In [22]:
DATASET_SIZE = subreddits.shape[0]

train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)

full_dataset = utils.build('reddit_all.tfrecord', True)
full_dataset = full_dataset.shuffle(seed=20200705, buffer_size=2**20)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

datasets = {'train': train_dataset, 'eval': val_dataset, 'test': test_dataset}

In [24]:
for key, dataset in datasets.items():
    filename = f"reddit_{key}.tfrecord"
    write_tfrecord_file(dataset, filename)

# Split into subreddits

In [101]:
top50 = ['AskReddit',
 'relationships',
 'leagueoflegends',
 'tifu',
 'relationship_advice',
 'trees',
 'gaming',
 'atheism',
 'AdviceAnimals',
 'funny',
 'politics',
 'pics',
 'sex',
 'WTF',
 'explainlikeimfive',
 'todayilearned',
 'Fitness',
 'IAmA',
 'worldnews',
 'DotA2',
 'TwoXChromosomes',
 'videos',
 'DestinyTheGame',
 'reddit.com',
 'offmychest',
 'buildapc',
 'AskMen',
 'personalfinance',
 'summonerschool',
 'technology',
 'wow',
 'NoFap',
 'starcraft',
 'dating_advice',
 'askscience',
 'Games',
 'news',
 'talesfromtechsupport',
 'depression',
 'pcmasterrace',
 'Guildwars2',
 'magicTCG',
 'loseit',
 'GlobalOffensive',
 'electronic_cigarette',
 'movies',
 'self',
 'Advice',
 'Drugs',
 'teenagers']


def filter_subreddit(record, subreddit):
#     print(records)
#     mask = tf.equal(records['topics'], tf.constant(subreddit, tf.string))
#     print(mask)
#     for c in records:
#         records[c] = tf.boolean_mask(records[c], mask)
#     return records
    return record['topics'] == subreddit

In [103]:
for key in ('test', 'eval', 'train'):
    dataset = build(f'reddit_{key}.tfrecord', True)
    dataset.cache()
    for subreddit in top50:
        sub_r = dataset.filter(lambda record: filter_subreddit(record, subreddit))
        filename = f'subreddits_data/subreddit_{subreddit}_{key}.tfrecord'
        writer = tf.data.experimental.TFRecordWriter(filename)
        write_tfrecord_file(sub_r, filename)